In [2]:
import polars as pl
import datetime
import holidays

dict = holidays.BE()

def is_weekend(date: datetime):
    return date.weekday() >= 5

def is_holiday(date: datetime):
    return date.date() in dict


In [3]:
# we want to figure out which non-weekend or holiday days attracted the most visitors.
(
    pl
        .read_parquet("../data/visitors.parquet")
        .filter(pl.col("timestamp").map_elements(lambda dt: not is_weekend(dt) and not is_holiday(dt), return_dtype = bool))
        .sort("visitors", descending = True)
        .limit(10)

)

timestamp,visitors
datetime[μs],i16
2020-08-18 18:00:00,2400
2023-12-27 18:00:00,2399
2023-01-30 18:00:00,2398
2022-08-01 18:00:00,2397
2023-08-31 18:00:00,2397
2024-01-29 18:00:00,2397
2020-08-28 18:00:00,2396
2020-09-23 18:00:00,2396
2022-02-22 18:00:00,2396


In [21]:
import os

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import BooleanType
 
os.environ["JAVA_HOME"] = "/home/gitpod/.sdkman/candidates/java/current"

spark = SparkSession.builder.getOrCreate()

def isNormalDay(dt: datetime):
    #print(str(dt) + " " +str(not is_weekend(dt) and not is_holiday(dt)) )
    return not is_weekend(dt) and not is_holiday(dt)

(
    spark
        .read
        .parquet("../data/visitors.parquet")
        .withColumn("isNormalDay", F.udf(isNormalDay, returnType="boolean")("timestamp"))
        .filter(F.col("isNormalDay"))
        .drop("isNormalDay")
        .sort("visitors", ascending = False)
        .limit(10)
        .show()
)

(
    spark
        .read
        .parquet("../data/visitors.parquet")
        .filter(F.udf(isNormalDay, returnType="boolean")("timestamp"))
        .sort("visitors", ascending = False)
        .limit(10)
        .show()
)

+-------------------+--------+
|          timestamp|visitors|
+-------------------+--------+
|2020-08-18 18:00:00|    2400|
|2023-12-27 18:00:00|    2399|
|2023-01-30 18:00:00|    2398|
|2022-08-01 18:00:00|    2397|
|2023-08-31 18:00:00|    2397|
|2024-01-29 18:00:00|    2397|
|2022-02-22 18:00:00|    2396|
|2020-08-28 18:00:00|    2396|
|2020-09-23 18:00:00|    2396|
|2024-03-14 18:00:00|    2396|
+-------------------+--------+

+-------------------+--------+
|          timestamp|visitors|
+-------------------+--------+
|2020-08-18 18:00:00|    2400|
|2023-12-27 18:00:00|    2399|
|2023-01-30 18:00:00|    2398|
|2022-08-01 18:00:00|    2397|
|2023-08-31 18:00:00|    2397|
|2024-01-29 18:00:00|    2397|
|2022-02-22 18:00:00|    2396|
|2020-08-28 18:00:00|    2396|
|2020-09-23 18:00:00|    2396|
|2024-03-14 18:00:00|    2396|
+-------------------+--------+



In [27]:
import datetime

test = holidays.BE()

print(test)

datetime.date(2024,5,1) in test

print(test)

{'country': BE, 'expand': True, 'language': None, 'market': None, 'observed': True, 'subdiv': None, 'years': set()}
{datetime.date(2024, 1, 1): "New Year's Day", datetime.date(2024, 3, 31): 'Easter Sunday', datetime.date(2024, 4, 1): 'Easter Monday', datetime.date(2024, 5, 1): 'Labor Day', datetime.date(2024, 5, 9): 'Ascension Day', datetime.date(2024, 5, 19): 'Whit Sunday', datetime.date(2024, 5, 20): 'Whit Monday', datetime.date(2024, 7, 21): 'National Day', datetime.date(2024, 8, 15): 'Assumption Day', datetime.date(2024, 11, 1): "All Saints' Day", datetime.date(2024, 11, 11): 'Armistice Day', datetime.date(2024, 12, 25): 'Christmas Day'}
